<a href="https://colab.research.google.com/github/sYanXO/NSE-meanReversion-strategy/blob/main/Mean_Reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- INSTALL AND IMPORT LIBRARIES ---
!pip install ta
!pip install yfinance

import yfinance as yf
import pandas as pd
import ta
import matplotlib.pyplot as plt

# --- STEP 1: DEFINE PORTFOLIO AND PARAMETERS ---

# A large sample of tickers from major NSE indices
tickers = [
    "RELIANCE.NS", "ADANIENT.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "LT.NS", "AXISBANK.NS", "BHARTIARTL.NS", "HINDUNILVR.NS", "KOTAKBANK.NS",
    "ITC.NS", "SBIN.NS", "MARUTI.NS", "BAJFINANCE.NS", "ASIANPAINT.NS",
    "TITAN.NS", "WIPRO.NS", "NESTLEIND.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "POWERGRID.NS", "TATAMOTORS.NS", "TECHM.NS", "BRITANNIA.NS", "ONGC.NS",
    "JSWSTEEL.NS", "HCLTECH.NS", "GRASIM.NS", "EICHERMOT.NS", "HEROMOTOCO.NS",
    "ADANIPORTS.NS", "COALINDIA.NS", "APOLLOHOSP.NS", "CIPLA.NS", "DRREDDY.NS",
    "INDUSINDBK.NS", "M&M.NS", "SBILIFE.NS", "TATASTEEL.NS", "TATACONSUM.NS",
    "HDFCBANK.NS", "ADANIENSOL.NS", "ADANIGREEN.NS", "ASIANPAINT.NS",
    "COFORGE.NS", "GODREJCP.NS", "HINDALCO.NS", "INFY.NS", "LTIM.NS",
    "PIDILITIND.NS", "PIIND.NS", "RECLTD.NS", "SBICARD.NS", "SIEMENS.NS",
    "SRF.NS", "SYNGENE.NS", "TATACOMM.NS", "TVSMOTOR.NS", "UNIONBANK.NS",
    "VOLTAS.NS"
]

start_date = "2020-01-01"
end_date = "2024-07-31"

# The optimal parameters we found through our iterative process
window = 20
long_term_window = 100
atr_multiple = 3.0
trend_filter_threshold = 0.04
nifty_ma_window = 200

# A list to store the returns from each individual trade across all stocks
all_portfolio_returns = []

# --- STEP 2: DOWNLOAD NIFTY 50 DATA FOR MARKET FILTER ---
try:
    nifty_df = yf.download('^NSEI', start=start_date, end=end_date, progress=False)
    nifty_df.columns = nifty_df.columns.droplevel(1)
    nifty_df['Nifty_MA'] = nifty_df['Close'].rolling(window=nifty_ma_window).mean()
    nifty_df.dropna(inplace=True)
    print("Nifty 50 data for market filter downloaded successfully.")
except Exception as e:
    print(f"Error downloading Nifty 50 data: {e}. Cannot proceed with market filter.")
    exit()

# --- STEP 3: MAIN BACKTESTING LOOP ---

for i, ticker in enumerate(tickers):
    # print(f"\n--- Backtesting {ticker} ({i+1}/{len(tickers)}) ---")

    # 3.1 Data Acquisition for the current ticker
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if df.empty:
            continue
        df.columns = df.columns.droplevel(1)
    except Exception as e:
        continue

    # 3.2 Define the "Mean", Deviation, and Indicators
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['StdDev'] = df['Close'].rolling(window=window).std()
    df['Upper_Band'] = df['SMA'] + (df['StdDev'] * 2)
    df['Lower_Band'] = df['SMA'] - (df['StdDev'] * 2)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['Long_SMA'] = df['Close'].rolling(window=long_term_window).mean()
    df['ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()

    # Merge Nifty data to filter on market trend
    # Merge Nifty data to filter on market trend
    # CORRECTED: Added lsuffix and rsuffix to handle overlapping 'Close' column
    df = df.join(nifty_df[['Nifty_MA', 'Close']], how='inner', lsuffix='_stock', rsuffix='_nifty')

    # CORRECTED: The rename operation now uses the new suffixes
    df.rename(columns={'Close_nifty': 'Nifty_Close', 'Close_stock': 'Close'}, inplace=True)
    df.dropna(inplace=True)

    # 3.3 Implement the Trading Logic
    in_long_position = False
    in_short_position = False
    long_buy_price = 0
    short_sell_price = 0

    for j in range(len(df)):
        current_date = df.index[j].date()
        close_price = df['Close'][j]
        upper_band = df['Upper_Band'][j]
        lower_band = df['Lower_Band'][j]
        sma = df['SMA'][j]
        long_sma = df['Long_SMA'][j]
        rsi = df['RSI'][j]
        atr = df['ATR'][j]
        nifty_ma = df['Nifty_MA'][j]
        nifty_close = df['Nifty_Close'][j]


        # Individual stock trend filter
        is_trending_up = close_price > long_sma * (1 + trend_filter_threshold)
        is_trending_down = close_price < long_sma * (1 - trend_filter_threshold)

        # Corrected Market-wide trend filter
        nifty_is_bullish = nifty_close > nifty_ma
        nifty_is_bearish = nifty_close < nifty_ma

        # --- Long Position Logic ---
        # Long Stop-Loss check
        if in_long_position and (close_price <= long_buy_price - (atr * atr_multiple)):
            sell_price = close_price
            profit = (sell_price - long_buy_price) / long_buy_price
            all_portfolio_returns.append(profit)
            in_long_position = False

        # Long Entry Condition: Price crosses below lower band, stock not in a downtrend, and Nifty is bullish
        elif not in_long_position and not in_short_position and not is_trending_down and nifty_is_bullish and close_price < lower_band and rsi < 30:
            in_long_position = True
            long_buy_price = close_price

        # Long Exit Condition: Price reverts to the mean (SMA)
        elif in_long_position and close_price >= sma:
            sell_price = close_price
            profit = (sell_price - long_buy_price) / long_buy_price
            all_portfolio_returns.append(profit)
            in_long_position = False

        # --- Short Position Logic ---
        # Short Stop-Loss check
        if in_short_position and (close_price >= short_sell_price + (atr * atr_multiple)):
            buy_to_cover_price = close_price
            profit = (short_sell_price - buy_to_cover_price) / short_sell_price
            all_portfolio_returns.append(profit)
            in_short_position = False

        # Short Entry Condition: Price crosses above upper band, stock not in an uptrend, and Nifty is bearish
        elif not in_long_position and not in_short_position and not is_trending_up and nifty_is_bearish and close_price > upper_band and rsi > 70:
            in_short_position = True
            short_sell_price = close_price

        # Short Exit Condition: Price reverts to the mean (SMA)
        elif in_short_position and close_price <= sma:
            buy_to_cover_price = close_price
            profit = (short_sell_price - buy_to_cover_price) / short_sell_price
            all_portfolio_returns.append(profit)
            in_short_position = False

    # Optional: Close any open positions at the end of the backtest
    if in_long_position:
        sell_price = df['Close'].iloc[-1]
        profit = (sell_price - long_buy_price) / long_buy_price
        all_portfolio_returns.append(profit)

    if in_short_position:
        buy_to_cover_price = df['Close'].iloc[-1]
        profit = (short_sell_price - buy_to_cover_price) / short_sell_price
        all_portfolio_returns.append(profit)

# --- STEP 4: EVALUATE PORTFOLIO RETURNS ---

print("\n\n--- Portfolio Backtest Complete ---")
if all_portfolio_returns:
    total_trades = len(all_portfolio_returns)
    cumulative_return = (pd.Series(all_portfolio_returns) + 1).prod() - 1
    win_rate = sum(r > 0 for r in all_portfolio_returns) / total_trades

    print(f"Total trades across all stocks: {total_trades}")
    print(f"Portfolio Cumulative Return: {cumulative_return:.2%}")
    print(f"Portfolio Win Rate: {win_rate:.2%}")
else:
    print("No trades were executed across the entire portfolio.")

/tmp/ipython-input-1950257282.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty_df = yf.download('^NSEI', start=start_date, end=end_date, progress=False)
/tmp/ipython-input-1950257282.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date, progress=False)
/tmp/ipython-input-1950257282.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-1950257282.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  upper_band = df['Upper_Band'][j]
/tmp/ipyt

Nifty 50 data for market filter downloaded successfully.


/tmp/ipython-input-1950257282.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_price = df['Close'][j]
/tmp/ipython-input-1950257282.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  upper_band = df['Upper_Band'][j]
/tmp/ipython-input-1950257282.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lower_band = df['Lower_Band'][j]
/tmp/ipython-input-1950257282.py:96: FutureWarning: Series.__getitem__ treating keys as posi



--- Portfolio Backtest Complete ---
Total trades across all stocks: 40
Portfolio Cumulative Return: 28.47%
Portfolio Win Rate: 70.00%
